In [ ]:
! pip install transformers trl peft accelerate datasets bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
  Attempting uninstal

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Pretraining & Instruction Tuning[SFT]

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from accelerate import Accelerator
import os

In [ ]:
train_config = {
            "model_ckpt": "meta-llama/Meta-Llama-3-8B-Instruct",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "sft-chatbot",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5,
            "is_pretraining": True
        }

In [ ]:
class Trainerr:

    def __init__(self, data, config):
        self.data = data
        self.config = config

    def prepare_lora_model(self):

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)
    def load_model_tokenizer(self):

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

        if self.config.is_pretraining:
            model_config = AutoConfig.from_pretrained(
                                                    self.config["model_ckpt"],
                                                    vocab_size=len(tokenizer),
                                                    n_ctx=self.config["max_seq_length"],
                                                    bos_token_id=tokenizer.bos_token_id,
                                                    eos_token_id=tokenizer.eos_token_id,
                                                )
            ## Add the configurations for changing the number of layers, heads
            self.model = AutoModelForCausalLM.from_config(model_config)
        else:
            self.model = AutoModelForCausalLM.from_pretrained(
                                self.config["model_ckpt"],
                                load_in_4bit=self.config["load_in_4bit"],
                                device_map=self.config["device_map"],
                                torch_dtype=self.config["torch_dtype"]
                            )
            self.model.config.use_cache=False
            self.model.config.pretraining_tp=1
            self.model = prepare_model_for_kbit_training(self.model)
            if self.config["use_lora"]:
                self.prepare_lora_model()



    def set_training_args(self):

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    # max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"]
                                )

    def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    peft_config=self.lora_config,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )
        else:
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )

    def train_and_save_model(self):
        self.create_trainer()
        self.trainer.train()
        self.trainer.save_model(self.config["output_dir"])
        self.tokenizer.save_pretrained(self.config["output_dir"])
        self.model.push_to_hub_gguf("Vasanth/"self.config["output_dir"], tokenizer, quantization_method = "q4_k_m", token = "")

In [ ]:
def formatting_func(context, question, answer):
    template = f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\n\n + \
    You must output the SQL query that answers the question.\n\n + \
    ### Input:\n + \
    ```{question}```\n\n + \
    ### Context:\n + \
    ```{context}```\n\n + \
    ### Response:\n + \
    ```{answer};```"""
    return template

In [ ]:
def create_data():
    data = load_dataset("HuggingFaceFW/fineweb", split="train")
    # data_df = data.to_pandas()
    # data_df = data_df[:5000]
    # data_df["text"] = data_df[["input", "instruction", "output"]].apply(lambda x: "Human: " + x["instruction"] + " " + x["input"] + " Assistant: "+ x["output"], axis=1)
    # data = Dataset.from_pandas(data_df)
    return data

data = create_data()

In [ ]:
train_sft = Trainerr(data, sft_config)
train_sft.train_and_save_model()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940290959023318
None


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Step,Training Loss
100,1.862600
200,1.604100


events.out.tfevents.1708005247.1b521e588a71.9936.0:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

# DPO

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import DPOTrainer
import pandas as pd
from accelerate import Accelerator
from peft import get_peft_model, LoraConfig

In [ ]:
dpo_config = {
            "model_ckpt": "Vasanth/mistral-chatbot",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 8,
            "lora_alpha": 8,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "mistral-chatbot-dpo",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 100,
            "num_train_epochs": 1,
            "max_steps": 250,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "neftune_noise_alpha": 5,
            "beta": 0.1,
            "loss_type": "kto-pair",
            "max_length": 768,
            "max_prompt_length": 512,
            "max_target_length": 256,
            "is_encoder_decoder": False
        }

In [ ]:
class TrainDPO:

    def __init__(self, data, config):
        self.data = data
        self.config = config

    def prepare_lora_model(self):

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)
        self.model_ref = get_peft_model(self.model_ref, self.lora_config)

    def load_model_tokenizer(self):

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )

        self.model_ref = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        # self.model = prepare_model_for_kbit_training(self.model)
        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def set_training_args(self):

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"],
                                    neftune_noise_alpha=self.config["neftune_noise_alpha"]
                                )

    def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )
        else:
            self.trainer = DPOTrainer(
                                        self.model,
                                        self.model_ref,
                                        peft_config=self.lora_config,
                                        args=self.set_training_args(),
                                        train_dataset=self.data,
                                        tokenizer=self.tokenizer,
                                        beta=self.config["beta"],
                                        loss_type=self.config["loss_type"],
                                        max_length=self.config["max_length"],
                                        max_prompt_length=self.config["max_prompt_length"],
                                        max_target_length=self.config["max_target_length"],
                                        is_encoder_decoder=self.config["is_encoder_decoder"]
                                    )

    def train_and_save_model(self):
        self.create_trainer()
        self.trainer.train()
        self.trainer.save_model(self.config["output_dir"])
        self.tokenizer.save_pretrained(self.config["output_dir"])

In [ ]:
def create_data():
    df = load_dataset("Anthropic/hh-rlhf", split="train").to_pandas()
    df["prompt"] = df["chosen"].apply(lambda x: x.split("Assistant: ")[0])
    df["chosen"] = df["chosen"].apply(lambda x: "Assistant: "+ x.split("Assistant: ")[-1])
    df["rejected"] = df["rejected"].apply(lambda x: "Assistant: " + x.split("Assistant: ")[-1])
    df = df.sample(1000)
    data = Dataset.from_pandas(df)
    return data
data = create_data()

In [ ]:
train_dpo = TrainDPO(data, dpo_config)
train_dpo.train_and_save_model()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.04703666202518836
None


/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:328: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
